Yahoo Finance request

In [ ]:
%pip install yfinance

In [ ]:
import yfinance as yf
fx = yf.Ticker("EURUSD=X")

In [50]:
import pandas as pd
fxHist = fx.history(start="2023-03-10", end="2023-03-11", interval= "1h")

for i in range(1, fxHist.shape[0]):
    result = fxHist.iloc[i][3] / fxHist.iloc[i-1][3]

result 

#resultDf = pd.DataFrame(result)
#resultDf


#fxHist.iloc[0][3] #1.060108


1.0

In [36]:
fxHist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-03-10 00:00:00+00:00,1.058873,1.060108,1.058873,1.060108,0,0.0,0.0
2023-03-10 01:00:00+00:00,1.060220,1.060220,1.059210,1.059771,0,0.0,0.0
2023-03-10 02:00:00+00:00,1.059771,1.059883,1.058425,1.059210,0,0.0,0.0
2023-03-10 03:00:00+00:00,1.059098,1.059996,1.058985,1.059883,0,0.0,0.0
2023-03-10 04:00:00+00:00,1.059996,1.060108,1.059771,1.059771,0,0.0,0.0
2023-03-10 05:00:00+00:00,1.059771,1.059771,1.058873,1.058985,0,0.0,0.0
2023-03-10 06:00:00+00:00,1.058873,1.059322,1.058425,1.058649,0,0.0,0.0
2023-03-10 07:00:00+00:00,1.058985,1.060670,1.058985,1.059771,0,0.0,0.0
2023-03-10 08:00:00+00:00,1.059883,1.060108,1.057753,1.059547,0,0.0,0.0


In [ ]:
fxDict = fxHist.to_json(orient='index') ##convertendo resultado em json
print(fxDict)

Conectando com cosmoDB-MS

In [ ]:
%pip install azure-cosmos

In [18]:
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import datetime
import config

In [19]:
#configs do cosmodb

HOST = config.settings['host']
MASTER_KEY = config.settings['master_key']
DATABASE_ID = config.settings['database_id']
CONTAINER_ID = config.settings['container_id']

In [20]:
# modelo de insert gerado pela ducumentação do Azure

def get_sales_order(item_id):
    order1 = {'id' : item_id,
            'partitionKey' : 'Account1',
            'purchase_order_number' : 'PO18009186470',
            'order_date' : datetime.date(2005,1,10).strftime('%c'),
            'subtotal' : 419.4589,
            'tax_amount' : 12.5838,
            'freight' : 472.3108,
            'total_due' : 985.018,
            'items' : [
                {'order_qty' : 1,
                    'product_id' : 100,
                    'unit_price' : 418.4589,
                    'line_price' : 418.4589
                }
                ],
            'ttl' : 60 * 60 * 24 * 30
            }

    return order1


def create_items(container):
    print('\nCreating Items\n')
    sales_order = get_sales_order("SalesOrder1")
    container.create_item(body=sales_order)

In [27]:
def run_sample():
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

        create_items(container)

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")

In [28]:
run_sample()

Database with id 'ToDoList' was found
Container with id 'Items' was found

Creating Items


run_sample done
